In [60]:
#importing some useful packages
import pickle
import cv2
import math
import time
import h5py
import json
import os
import csv
import random
import numpy as np
import pandas as pd
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import theano
%matplotlib inline

In [61]:
df = pd.read_csv('driving_log_Arun.csv')
print(len(df))
#if (df['center'].values[0][16:18] == '12'):
#    print ('yes')
#print(df[df['center'].values[0][16:18] == '12'])
df = df.drop(df[df['steering'] == 0].sample(frac=0.60).index)
#repeat = 4
#df = pd.concat(df[df['center'].values[16:18] == '02']*repeat)
center = df[[0]]

center = np.squeeze(center.values)
print(center[0][16:18])
left = df[[1]]
left = np.squeeze(left.values)
right = df[[2]]
right = np.squeeze(right.values)
steering = df[[3]]
steering = np.squeeze(steering.values)
correction = 0.25
steering_left = steering + correction
steering_right = steering - correction
print(type(center))

images = np.concatenate((np.concatenate((center, left)), right))
steer = np.concatenate((np.concatenate((steering, steering_left)), steering_right))
print(images.shape)
print (steer.shape)
batch_size = 10

def remove_spaces(path):
    len_path = len(path)
    for i in range(len_path):
        if (path[i][0] == ' '):
            path[i] = path[i][1:]
    return path

images = remove_spaces(images)
row_size = 30
col_size = 120
ch = 3
#print(images[])

10333
12
<class 'numpy.ndarray'>
(20418,)
(20418,)


In [62]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    horizontal_flip=True)



In [63]:
def data_generator(Images,steering,training=0,batch_size=10,threshold=0.25):
    ctr=0

    while( (training and ctr<len(steering) ) or (not training) ) :
    
        if ((not training) and (ctr>=len(steering))):
            ctr=0

        data,labels=preprocess(Images[ctr:ctr+batch_size],steering[ctr:ctr+batch_size],threshold)    
        ctr +=batch_size

        yield data,labels
        
def crop(img):
    #new_size_col=120
    #new_size_row=30
    img=img[60:140,:,:]
    img=cv2.resize(img,(col_size,row_size), interpolation=cv2.INTER_AREA)
    return img
        
def preprocess(Images,steering,threshold):
    row,col,ch=(mpimg.imread(Images[0])).shape

    img = []
    steer = []
        
    for i,path in enumerate(Images):

        I=mpimg.imread(path)
        I = I/255 - 0.5
        #if(np.random.randint(2)):
        #    I=augment_brightness_camera_images(I)

            
        img.append(crop(I))
        steer.append(steering[i])

        if steering[i]<-threshold or steering[i]>threshold :
            I=np.fliplr(I)
            img.append(crop(I))
            steer.append(-steering[i])

    return np.asarray(img),np.asarray(steer)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(images, steer, test_size = 0.2, random_state = 0)

train_data = data_generator(X_train, y_train)
batch_x, batch_y = next(train_data)
print(batch_y)
batch_x, batch_y = next(train_data)
print(batch_y)
"""
X = np.zeros(shape = (len(X_train), row_size, col_size, 3))
for i in range(len(X_train)):
    img = mpimg.imread(X_train[i])
    img = img/255 - 0.5
    X[i] = crop(img)
Xtest = np.zeros(shape = (len(X_test), row_size, col_size, 3))
for i in range(len(X_test)):
    img = mpimg.imread(X_test[i])
    img = img/255 - 0.5
    Xtest[i] = crop(img)
"""

[-0.4141951   0.4141951  -0.4141951   0.4141951   0.1275214   0.26391724
 -0.26391724 -0.03891354  0.          0.1287396  -0.3952064   0.3952064
  0.25       -0.5755992   0.5755992 ]
[ 0.0904655  -0.0160065  -0.05975719  0.4457194  -0.4457194  -0.25       -0.25
  0.         -0.25       -0.25        0.32132844 -0.32132844]


'\nX = np.zeros(shape = (len(X_train), row_size, col_size, 3))\nfor i in range(len(X_train)):\n    img = mpimg.imread(X_train[i])\n    img = img/255 - 0.5\n    X[i] = crop(img)\nXtest = np.zeros(shape = (len(X_test), row_size, col_size, 3))\nfor i in range(len(X_test)):\n    img = mpimg.imread(X_test[i])\n    img = img/255 - 0.5\n    Xtest[i] = crop(img)\n'

In [65]:
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Convolution2D, Flatten, MaxPooling2D, Dropout

model = Sequential()
#model.add(Lambda(lambda x: x/255 - 0.5, input_shape=(row_size, col_size, 3)))
model.add(Convolution2D(64, 3, 3, border_mode = 'valid', input_shape=(row_size, col_size, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Convolution2D(128, 3, 3))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('elu'))
model.add(Dense(10))
model.add(Activation('elu'))
model.add(Dropout(0.5))

model.add(Dense(1))
#model.add(Activation('softmax'))

In [66]:
model.summary()

'''
model.compile(optimizer='rmsprop',
          loss='binary_crossentropy',
          metrics=['accuracy'])
'''

model.compile(loss='mse',
              optimizer=Adam(lr = 0.001),
              metrics=['mean_squared_error'])
"""
history = model.fit(data_generator(X_train,y_train,training=0,batch_size=128),
                    samples_per_epoch=len(y_train),
                    validation_data=data_generator(X_test,y_test,training=0,batch_size=128),
                    nb_val_samples=len(y_test), nb_epoch=10, verbose=1)
"""
model.fit_generator(data_generator(X_train,y_train,training=0,batch_size=64),
                    samples_per_epoch=len(y_train),
                    validation_data=data_generator(X_test,y_test,training=0,batch_size=64),
                    nb_val_samples=len(y_test),                    
                    nb_epoch=15,
                    verbose=2)
"""
model.fit_generator(datagen.flow(X, y_train, batch_size=128),
                    samples_per_epoch=len(X_train), validation_data = datagen.flow(Xtest, y_test, batch_size=128),
                    nb_val_samples = len(Xtest), nb_epoch = 15, verbose = 2)
"""

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_22 (Convolution2D) (None, 28, 118, 64)   1792        convolution2d_input_7[0][0]      
____________________________________________________________________________________________________
activation_36 (Activation)       (None, 28, 118, 64)   0           convolution2d_22[0][0]           
____________________________________________________________________________________________________
maxpooling2d_22 (MaxPooling2D)   (None, 14, 59, 64)    0           activation_36[0][0]              
____________________________________________________________________________________________________
convolution2d_23 (Convolution2D) (None, 12, 57, 64)    36928       maxpooling2d_22[0][0]            
___________________________________________________________________________________________

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


52s - loss: 0.1276 - mean_squared_error: 0.1276 - val_loss: 0.0399 - val_mean_squared_error: 0.0399
Epoch 2/15
45s - loss: 0.0509 - mean_squared_error: 0.0509 - val_loss: 0.0366 - val_mean_squared_error: 0.0366
Epoch 3/15
45s - loss: 0.0436 - mean_squared_error: 0.0436 - val_loss: 0.0313 - val_mean_squared_error: 0.0313
Epoch 4/15
45s - loss: 0.0419 - mean_squared_error: 0.0419 - val_loss: 0.0272 - val_mean_squared_error: 0.0272
Epoch 5/15
45s - loss: 0.0397 - mean_squared_error: 0.0397 - val_loss: 0.0303 - val_mean_squared_error: 0.0303
Epoch 6/15
45s - loss: 0.0375 - mean_squared_error: 0.0375 - val_loss: 0.0236 - val_mean_squared_error: 0.0236
Epoch 7/15
46s - loss: 0.0370 - mean_squared_error: 0.0370 - val_loss: 0.0270 - val_mean_squared_error: 0.0270
Epoch 8/15
45s - loss: 0.0357 - mean_squared_error: 0.0357 - val_loss: 0.0267 - val_mean_squared_error: 0.0267
Epoch 9/15
45s - loss: 0.0350 - mean_squared_error: 0.0350 - val_loss: 0.0250 - val_mean_squared_error: 0.0250
Epoch 10/15


'\nmodel.fit_generator(datagen.flow(X, y_train, batch_size=128),\n                    samples_per_epoch=len(X_train), validation_data = datagen.flow(Xtest, y_test, batch_size=128),\n                    nb_val_samples = len(Xtest), nb_epoch = 15, verbose = 2)\n'

In [67]:
model.save('model.h5')

In [2]:
#reading in an image
log = pd.read_csv('driving_log.csv')
X = []
print(len(log))
for n in range(100):
    img = mpimg.imread(log.ix[n][0][27:])
    left = mpimg.imread(log.ix[n][1][28:])
    right = mpimg.imread(log.ix[n][2][28:])
    new = np.concatenate((np.concatenate((left,img), axis = 2), right), axis = 2)
    new = new/255 - 0.5
    X.append(new)
X = np.asarray(X)
X_val = X
print(X.shape)
Y = np.asarray(log.ix[:99,3])
Y_val = Y
print(Y.shape)

1861
(100, 160, 320, 9)
(100,)


In [28]:
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Convolution2D, Flatten, MaxPooling2D, Dropout

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode = 'valid', input_shape = (160, 320, 9)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('softmax'))

In [29]:
model.summary()

model.compile(optimizer='rmsprop',
          loss='binary_crossentropy',
          metrics=['accuracy'])
history = model.fit(X, Y, 
                    batch_size=32, nb_epoch=10, verbose=1, 
                    validation_data=(X_val,Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_10 (Convolution2D) (None, 158, 318, 32)  2624        convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 158, 318, 32)  0           convolution2d_10[0][0]           
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 79, 159, 32)   0           activation_13[0][0]              
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 77, 157, 64)   18496       maxpooling2d_7[0][0]             
___________________________________________________________________________________________

KeyboardInterrupt: 